
# TensorFlow基本操作
```
Author: PeterSansan
Project: https://github.com/PeterSansan/TF_ACTION
```

### [目录]
- [1、算法运算比较GPU与CPU的性能](#1) 
- [2、另一种会话形式](#2)
- [3、assign赋值函数](#3)
- [4、fetch取数据 ](#4)
- [5、feed填数据](#5) 
- [6、get_variable与Variable的一点区别](#6)
- [7、argmax](#7)
- [8、dropout](#8)
- [9、保存数据](#9)
- [10、读取数据](#10)
- [11、变量的赋值不要用run](#11)
- [12、print输出到文件](#12)
- [13、初始化函数](#13)
- [14、命名域与共享变量](#14)
- [15、查看CPU与GPU使用情况](#15)

In [1]:
# -*- coding:utf-8 -*-
from __future__ import print_function

import tensorflow as tf
import numpy as np
import time
import sys

### <span id="1">1.乘法运算比较GPU与CPU的性能</span>

In [2]:
begin=time.time() # 时间开始
with tf.Session() as sess0:
    with tf.device("/gpu:0"):#g 不加这个话就不能控制用CPU还是GPU，默认是GPU
        matrix1 = np.random.rand(1000,1500).astype(np.float32)
        matrix2 = np.random.rand(1500,1000).astype(np.float32)
        product = tf.matmul(matrix1, matrix2)
        result = sess0.run(product)

end = time.time()  # 时间结束
print("Spend time %f s" %(end - begin))

Spend time 0.206182 s


In [3]:
begin=time.time() # 时间开始
with tf.Session() as sess0:
    with tf.device("/cpu:0"):
        matrix1 = np.random.rand(1000,1500).astype(np.float32)
        matrix2 = np.random.rand(1500,1000).astype(np.float32)
        product = tf.matmul(matrix1, matrix2)
        result = sess0.run(product)

end = time.time()  # 时间结束
print("Spend time %f s" %(end - begin))

Spend time 0.564962 s


上面跑出来的结果可以看出GPU比CPU要快一些

### <span id="2">2.另一种会话形式</span>
前面那种会话是在模型与数据准备好的情况下，开启会话的，一般用`with tf.Session() as sess:`包含在里面，其实还有另外一种交互式更多好的会话形式。

In [4]:
#deploy a session
sess = tf.InteractiveSession()

#design the grape
matrix1 = np.random.rand(200,150).astype(np.float32)
matrix2 = np.random.rand(150,200).astype(np.float32)
product = tf.matmul(matrix1, matrix2)

#run the operation
print(product.eval())

sess.close()

[[ 36.63120651  35.20991898  35.93156815 ...,  33.42737198  35.38289261
   31.6554718 ]
 [ 42.56116486  42.72803116  41.04106522 ...,  40.08535385  43.54841614
   40.63188934]
 [ 38.30940628  36.58013535  38.64689636 ...,  32.69615173  37.78046036
   36.85554504]
 ..., 
 [ 39.15103149  38.87597656  40.16497421 ...,  37.36235428  40.66197968
   36.84383774]
 [ 39.997509    37.88767242  36.65858459 ...,  34.99864578  38.94895172
   36.97928619]
 [ 38.40794754  40.0220871   38.5599556  ...,  38.88759232  40.09415817
   35.58378601]]


### <span id="3">3.assign赋值函数</span>
实现了计算器功能

In [5]:
#design the graph
state = tf.Variable(0, name="counter")

one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)  #赋值

#initialization
init_op = tf.global_variables_initializer()

#run 
with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run(state))
    for _ in range(4):
        sess.run(update)
        print(sess.run(state))

0
1
2
3
4


### <span id="4">4.fetch取数据</span>
这种方法实际上我们上面一直在用

In [6]:
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)    # 定义三个常量

intermed = tf.add(input2, input3)
mul = tf.mul(input1, intermed)

with tf.Session() as sess:
    result = sess.run([mul, intermed])
    print(result)  # 取两个结果
    result = sess.run([intermed])
    print(result)   # 取一个结果

[21.0, 7.0]
[7.0]


### <span id="5">5.feed填数据</span>

In [7]:
input1 = tf.placeholder(tf.float32,shape=(3, 3))
input2 = tf.placeholder(tf.float32,shape=(3, 3))
output = tf.matmul(input1, input2)#matmul is different mul

with tf.Session() as sess:
    rand_array = np.ones([3, 3])
    print(sess.run([output], feed_dict={input1: rand_array,input2: rand_array}))

[array([[ 3.,  3.,  3.],
       [ 3.,  3.,  3.],
       [ 3.,  3.,  3.]], dtype=float32)]


### <span id="6">6.get_variable与Variable的一点区别</span>

In [8]:

from pprint import pprint

a = tf.get_variable('a',shape=[5,2])    # 默认 trainable=True
b = tf.get_variable('b',shape=[2,5],trainable=False)
c = tf.constant([1,2,3],dtype=tf.int32,shape=[8],name='c') # 因为是常量，所以trainable=False
d = tf.Variable(tf.random_uniform(shape=[3,3]),name='d')

tvar = tf.trainable_variables()
tvar_name = [x.name for x in tvar]
print(tvar)
# [<tensorflow.python.ops.variables.Variable object at 0x7f9c8db8ca20>, <tensorflow.python.ops.variables.Variable object at 0x7f9c8db8c9b0>]
print(tvar_name)
# ['a:0', 'd:0']

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
pprint(sess.run(tvar))

[<tensorflow.python.ops.variables.Variable object at 0x7f4cc8113590>, <tensorflow.python.ops.variables.Variable object at 0x7f4d14256150>, <tensorflow.python.ops.variables.Variable object at 0x7f4ccd1980d0>]
[u'counter:0', u'a:0', u'd:0']
[0,
 array([[ 0.18692321,  0.64265215],
       [ 0.14546293, -0.62066239],
       [-0.44651672,  0.35067594],
       [ 0.08697903,  0.24695468],
       [-0.0844515 ,  0.27527654]], dtype=float32),
 array([[ 0.15427423,  0.00526881,  0.58931553],
       [ 0.79344165,  0.46213388,  0.9489609 ],
       [ 0.89840972,  0.50520837,  0.96656311]], dtype=float32)]


### <span id="7">7.argmax</span>

In [9]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
aa = tf.get_variable(name = 'aa',
                    shape=[3,4],
                    dtype = tf.float32,
                    initializer=tf.random_uniform_initializer(minval=-1,maxval=1))

bb = tf.argmax(input = aa,dimension = 0) # 选出每列中最大值的位置
cc = tf.argmax(input = aa,dimension = 1) # 选出每行中最大值的位置

sess.run(tf.global_variables_initializer())
print(sess.run(aa))

[[-0.54428887 -0.73691559  0.8035264   0.9254961 ]
 [-0.44370866 -0.44143152 -0.07948303  0.07233262]
 [ 0.45271635  0.66048956  0.42163539  0.20828128]]


### <span id="8">8、dropout</span>

In [10]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
ai = tf.get_variable('ai',shape=[2,5])
bi = ai
ci = ai
a_drop = tf.nn.dropout(ai,0.8)
c_drop = tf.nn.dropout(ci,1)
sess.run(tf.global_variables_initializer())

print(sess.run(ai))
print(sess.run(a_drop))
print(sess.run(c_drop))

[[-0.34618318  0.62118101 -0.84818065  1.07047594 -1.0440985 ]
 [-0.82751012 -0.60918528 -0.09876704 -0.64386338 -0.47689581]]
[[-0.43272898  0.77647626 -0.          1.33809495 -1.30512309]
 [-1.03438759 -0.         -0.         -0.         -0.59611976]]
[[-0.34618318  0.62118101 -0.84818065  1.07047594 -1.0440985 ]
 [-0.82751012 -0.60918528 -0.09876704 -0.64386338 -0.47689581]]


### <span id="9">9、保存数据</span>

In [11]:
# save to file  
# 下面代码注释掉，因为一个文件保存与读取不要重复了
# W = tf.Variable([[1,2,3],[4,5,6]],dtype = tf.float32,name='weight')  
# b = tf.Variable([[1,2,3]],dtype = tf.float32,name='biases')  
  
# init = tf.global_variables_initializer()  
# saver = tf.train.Saver()  
# with tf.Session() as sess:  
#       sess.run(init)  
#       save_path = saver.save(sess,"./../codes/my_save_net/save_net.ckpt")  
#       print ("save to path:",save_path)  
#       print ("weights:",sess.run(W))  
#       print ("biases:",sess.run(b)) 

### <span id="10">10、加载数据</span>

In [12]:
tf.reset_default_graph()
W = tf.Variable(np.arange(6).reshape((2,3)),dtype = tf.float32,name='weight')  
b = tf.Variable(np.arange(3).reshape((1,3)),dtype = tf.float32,name='biases')  
  
saver = tf.train.Saver()  
with tf.Session() as sess:  
        saver.restore(sess,"./../codes/my_save_net/save_net.ckpt")  
        print ("weights:",sess.run(W))  
        print ("biases:",sess.run(b))  

weights: [[ 1.  2.  3.]
 [ 4.  5.  6.]]
biases: [[ 1.  2.  3.]]


### <span id="11">11、变量的赋值不要用run</span>

In [13]:
tf.reset_default_graph()
t1 = time.time()  
x = tf.Variable([1.0])  
b =1.0  
start1 = time.time()  
with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer())  
    #通过sess.run的方式读变量  
    for step in range(5000):  
        res = sess.run(x)  
    print("通过sess.run的方式读变量所需的时间:",time.time()-start1)  
    start2 = time.time()  
    for step in range(5000):  
        a = b  
    print("通过直接赋值的手段读变量所需的时间:",time.time()-start2)  

通过sess.run的方式读变量所需的时间: 1.02698397636
通过直接赋值的手段读变量所需的时间: 0.000610113143921


### <span id="12">12、print输出到文件</span>
这个功能不是TensorFlow的，放在这里只是它也很常用到

In [14]:
f=open('a.txt','a+') # 追加

old=sys.stdout #将当前系统输出储存到一个临时变量中
sys.stdout=f  #输出重定向到文件
print('Hello weird') #测试一个打印输出
sys.stdout.flush() # 刷新文件流
sys.stdout=old #还原原系统输出
f.close() 
print(open('a.txt','r').read())

# 第二种方法
# f=open('test.txt','a+')
# s= '123'
# abc= '456'
# print >> f, s,abc
# f.close()

Hello weird
Hello weird
Hello weird



In [15]:
ls

1_basic_operations.ipynb  a.txt  README.md


### <span id="13">13、使用初始化函数</span>

In [16]:
value = [0,1,2,3,4,5,6,7]
#value = np.array(value)  # 注释掉没有影响
#value = value.reshape([2,4])
init = tf.constant_initializer(value)
print('fitting shape:')
tf.reset_default_graph()
with tf.Session():
  x = tf.get_variable('x',shape=[2,4],initializer=init)
  x.initializer.run()
  print(x.eval())

fitting shape:
[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]]


In [17]:
print('larger shape:')  #小型的初始化是不允许的，也就是说shape小于原来的大小
tf.reset_default_graph()
with tf.Session():
  x = tf.get_variable('x',shape=[3,4],initializer = init)
  x.initializer.run()
  print(x.eval())

larger shape:
[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 7.  7.  7.  7.]]


In [18]:
print('random_normal_initializer:')
tf.reset_default_graph()
init = tf.random_normal_initializer(mean=0.0,stddev=1.0) #正态分布函数
with tf.Session():
  x = tf.get_variable('x',shape=[3,4],initializer = init)
  x.initializer.run()
  print(x.eval())

random_normal_initializer:
[[-0.43178701 -1.43916285 -0.48576009  2.22311687]
 [ 1.16909182  0.31709123 -0.82687312 -0.0599713 ]
 [-0.23779255  0.1306451  -1.19138658 -0.00984022]]


In [19]:
print('truncated_normal_initializer:')
tf.reset_default_graph()
init = tf.truncated_normal_initializer(mean=0.0,stddev=1.0)
with tf.Session():
  x = tf.get_variable('x',shape=[3,4],initializer = init)
  x.initializer.run()
  print(x.eval())

truncated_normal_initializer:
[[-0.27428246  0.31585512 -1.13757443  0.35627115]
 [ 0.64936376  0.21050541 -0.91876692  1.64121509]
 [ 1.06470704  0.19850279 -0.64019591  0.07522314]]


In [20]:
print('random_uniform_initializer:')
tf.reset_default_graph()
init = tf.random_uniform_initializer(minval=0,maxval=None) #均匀分布随机数
with tf.Session():
  x = tf.get_variable('x',shape=[3,4],initializer = init)
  x.initializer.run()
  print(x.eval())

random_uniform_initializer:
[[ 0.11234164  0.47092831  0.47490561  0.74645078]
 [ 0.73484349  0.49166548  0.96968091  0.50727308]
 [ 0.78294468  0.49954581  0.31730163  0.88692081]]


```
【说明】：
tf.random_normal | tf.truncated_normal | tf.random_uniform
tf.random_normal(shape,mean=0.0,stddev=1.0,dtype=tf.float32,seed=None,name=None)
tf.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
tf.random_uniform(shape,minval=0,maxval=None,dtype=tf.float32,seed=None,name=None)
这几个都是用于生成随机数tensor的。尺寸是shape
random_normal: 正太分布随机数，均值mean,标准差stddev
truncated_normal:截断正态分布随机数，均值mean,标准差stddev,不过只保留[mean-2*stddev,mean+2*stddev]范围内的随机数
random_uniform:均匀分布随机数，范围为[minval,maxval]
```

### <span id="14">14、命名域与共享变量</span>

In [21]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
# 不同的命名域与变量
with tf.variable_scope("foo"):
  with tf.variable_scope("bar"):
    v = tf.get_variable("v", [1])
    print(v.name)

print(v.name)

with tf.variable_scope("foo1"):
  with tf.variable_scope("bar1"):
    v = tf.get_variable("v", [1])
    print(v.name)
print(v.name)


foo/bar/v:0
foo/bar/v:0
foo1/bar1/v:0
foo1/bar1/v:0


In [22]:
# 变量共享（相同变量）
tf.reset_default_graph()
sess = tf.InteractiveSession()
with tf.variable_scope("xxx"):
  a = tf.get_variable("a",[1])
with tf.variable_scope("xxx",reuse=True):   # 采用xxx/a的值
  a1 = tf.get_variable("a",[1])
print(a,a1)
sess.run(tf.global_variables_initializer())
print(a.eval(),a1.eval()) #如果初始化为None,则会采用variable_scope的初始化值，
# 如果也是None,则采用uniform_unit_scaling_initializer
assert a==a1  # a , a1 是一样的

Tensor("xxx/a/read:0", shape=(1,), dtype=float32) Tensor("xxx/a/read:0", shape=(1,), dtype=float32)
[ 0.39663684] [ 0.39663684]


In [23]:
# 变量共享（相同变量）另一种写法
tf.reset_default_graph()
with tf.variable_scope("yyy") as scope:
    v = tf.get_variable("v", [1])
    scope.reuse_variables()
    v1 = tf.get_variable("v", [1])
assert v1 == v

In [24]:
# 为防止在没有使用reuse的情况下出现相现的共享变量，则会弹出异常,如下面是有错误的
# with tf.variable_scope("zzz"):
  # v = tf.get_variable("v",[1])
  # v1 = tf.get_variable("v",[1])
  # Raises ValueError("...v already exists ...").
  
# 为防止在使用reuse的情况下引用了之前没有的共享变量，则会弹出异常，如下面是有错误的
# with tf.variable_scope("aaa",reuse=True):
  # v = tf.get_variable("v",[1])
  # Raises ValueError("... v does not exists...").

### <span id="15">15.查看CPU与GPU使用</span>

```
CPU:
    sensors   or   watch sensors
```

```
GPU:
    nvidia-smi --query-gpu=temperature.gpu --format=csv,noheader
    or 
    nvidia-smi -a  # 可以看到显示的所有信息，包括什么温度会关自动关闭
    or
    watch -n 1 nvidia-smi  # 1秒的频率
```